In [1156]:
import pandas as pd

In [1157]:
class Quiz:
    def __init__(self, nome_quiz, total_questoes):
        self.nome_quiz = nome_quiz
        self.total_questoes = total_questoes
        self.acertos = 0
        self.erros = 0
        self.duracao_total = 0
        self.user = ''
        self.tentativa = 0
        self.resultados = {}

    
    # Função responsável por contabilizar os acertos, os erros e o tempo toda da partida
    def respostas (self, user='',  tentativa=0, resposta_correta=[], tempo_resposta=[]):

        self.tentativa = tentativa
        self.user = user
        self.duracao_total = sum(tempo_resposta)

        for resposta in resposta_correta:
            if resposta:
                self.acertos += 1
            else:
                self.erros += 1

        duracao_total = f"{(self.duracao_total/60):.2f} min"

        self.resultados[self.user,f"Tentativa {self.tentativa}"]= {
                'Quiz':             self.nome_quiz,
                'Acertos':          self.acertos,
                'Erros':            self.erros,
                'Duração total':    duracao_total
            }
        
        self.acertos = 0
        self.erros = 0
        
        data_resutados = pd.DataFrame(self.resultados).T

        return data_resutados
                  
    
    def avaliar_desempenho(self):

        # Calculando desempenho
        media_tempo_resposta = self.duracao_total / self.total_questoes
        percentual_acertos = (self.acertos / self.total_questoes) * 100

        # Avaliando o resultado do usuário
        if percentual_acertos >= 70 and media_tempo_resposta <= 20:
            mensagem = "APROVADO: Você atingiu o desempenho necessário para passar de fase."
        else:
            mensagem = "REPROVADO: Você não atingiu o desempenho necessário para passar de fase."

        # Resetando variáveis
        self.acertos = 0
        self.erros = 0
        self.duracao_total = 0
        
        return self.user,self.nome_quiz, self.tentativa, percentual_acertos, media_tempo_resposta, mensagem



In [1158]:
# INPUT INICIAL
nome_quiz       = 'Exame - hemograma'
total_questoes  = 10

quiz = Quiz(nome_quiz,total_questoes)

# Analise de cada tentativa

In [1159]:
# Função responsável por  filtrar as informações do DataFrame
def analise_tentativa (user, data, tentativa):

        # Filtrando o DataFrame
        data_filtrado =  data.loc[(data['User_ID'] == user) & (data['Tentativa'] == f'Tentativa_{tentativa}'), 'Respostas'].iloc[0]

        respostas = []
        tempo = []

        for questao in range(total_questoes):
                
                respostas.append(data_filtrado[questao][0])
                tempo.append(data_filtrado[questao][1])
        
        quiz.respostas(user, tentativa, respostas, tempo)

# Função responsável por apresentar as informações analizadas
def visualizar_desempenho ():

        # Buscando as informações da função avaliar_desempenho
        user_id, quiz_nome, tentativa, percentual_acertos, media_tempo_resposta, resultado = quiz.avaliar_desempenho()

        print(f"""
                QUIZ: {quiz_nome}

                Usuário .............................. {user_id}
                Tentativa realizada .................. {tentativa}° tentativa
                Percentual de acertos ................ {percentual_acertos:.2f}%
                Média do tempo de resposta ........... {media_tempo_resposta:.2f} segundos

                {resultado}
                """)
        

# Consulta

In [1160]:
def consulta (df):
    # user = str(input('Digite o user_ID do usuário: '))
    user = 'User_1'
    tentativas = df.loc[(df['User_ID'] == user)]

    for vez in range(1, len(tentativas) + 1):
        analise_tentativa(user, df, vez)
        
    data_user = quiz.respostas().loc[user]

    
    print(f" PROGRESSO DO USER: {user}")
    print(display(data_user))

# Analise do progresso

In [1161]:
def progresso (user):
    data_user = quiz.respostas().loc[user]
    user = 'User_1'

    df = quiz.respostas().loc[user]

    tentativas = len(df)

# Gerando DataFrame teste

In [1162]:
import pandas as pd
import random

# Função para gerar tentativas aleatórias
def gerar_tentativa():
    tentativa = []
    for _ in range(10):  # 10 questões no quiz
        resultado = random.choice([True, False])  # Escolhe aleatoriamente se a resposta está correta ou não
        tempo = random.randint(10, 30)  # Tempo aleatório entre 10 e 30 segundos
        tentativa.append((resultado, tempo))
    return pd.Series(tentativa)

# Dicionário para armazenar as respostas dos usuários
respostas = {}

# Gerar 10 novos usuários com 4 tentativas diferentes cada
for i in range(1, 11):
    user_id = f'User_{i}'
    tentativas_usuario = {}
    for j in range(1, 5):
        nome_tentativa = f'Tentativa_{j}'
        tentativa = gerar_tentativa()
        tentativas_usuario[nome_tentativa] = tentativa
    respostas[user_id] = tentativas_usuario

# Criar DataFrame com os resultados das tentativas dos usuários
data = []
for user_id, tentativas_usuario in respostas.items():
    for tentativa, respostas_tentativa in tentativas_usuario.items():
        data.append([user_id, tentativa, respostas_tentativa])

# Criar DataFrame a partir dos dados coletados
df = pd.DataFrame(data, columns=['User_ID', 'Tentativa', 'Respostas'])

# Mostrar DataFrame
display(df.head(5))

,User_ID,Tentativa,Respostas
0,User_1,Tentativa_1,"0 (False, 21) 1 (True, 20) 2 (True,..."
1,User_1,Tentativa_2,"0 (False, 13) 1 (False, 27) 2 (False,..."
2,User_1,Tentativa_3,"0 (True, 29) 1 (False, 17) 2 (False,..."
3,User_1,Tentativa_4,"0 (False, 20) 1 (True, 27) 2 (False,..."
4,User_2,Tentativa_1,"0 (False, 26) 1 (False, 28) 2 (True,..."


# Testes

In [1163]:
consulta(df)

 PROGRESSO DO USER: User_1


,Quiz,Acertos,Erros,Duração total
Tentativa 1,Exame - hemograma,6,4,3.67 min
Tentativa 2,Exame - hemograma,3,7,3.35 min
Tentativa 3,Exame - hemograma,4,6,3.20 min
Tentativa 4,Exame - hemograma,5,5,2.92 min


None


In [1164]:
user ='User_1'
data_filtrado = df.loc[(df['User_ID'] == 'User_1') & (df['Tentativa'] == 'Tentativa_2'), 'Respostas'].iloc[0]
data_filtrado


0    (False, 13)
1    (False, 27)
2    (False, 28)
3     (True, 28)
4    (False, 19)
5    (False, 17)
6    (False, 12)
7     (True, 25)
8    (False, 21)
9     (True, 11)
dtype: object